In [2]:
import pandas as pd
import psycopg2	
con = psycopg2.connect(dbname='mimic', user='postgres', password='23703513Tk' )
query_schema = 'set search_path to ' + 'mimiciii'+ ';'

In [3]:

pd.read_sql(query_schema+"""
SELECT *
FROM patients
LIMIT 5
""", con)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,234,249,F,2075-03-13,NaT,NaT,None,0
1,235,250,F,2164-12-27,2188-11-22,2188-11-22,None,1
2,236,251,M,2090-03-15,NaT,NaT,None,0
3,237,252,M,2078-03-06,NaT,NaT,None,0
4,238,253,F,2089-11-26,NaT,NaT,None,0


In [5]:
pd.read_sql(query_schema+"""
SELECT gender, count(*)
FROM patients
GROUP BY gender
""", con)

,gender,count
0,M,26121
1,F,20399


In [6]:
pd.read_sql(query_schema+"""
WITH first_admission_time AS
(
  SELECT
      p.subject_id, p.dob, p.gender, p.expire_flag
      , MIN (a.admittime) AS first_admittime
      , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age 
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender, p.expire_flag
  ORDER BY p.subject_id
)
, age as
(
  SELECT
      subject_id, dob, gender, expire_flag
      , first_admittime, first_admit_age
      , CASE
          WHEN first_admit_age > 100
              then '>89'
          WHEN first_admit_age >= 14
              THEN 'adult'
          WHEN first_admit_age <= 1
              THEN 'neonate'
          ELSE 'middle'
          END AS age_group
  FROM first_admission_time
)
select age_group, gender, expire_flag
  , count(subject_id) as NumberOfPatients
from age
group by age_group, gender, expire_flag
ORDER BY age_group, gender
""", con)

,age_group,gender,expire_flag,numberofpatients
0,>89,F,0,356
1,>89,F,1,938
2,>89,M,1,542
3,>89,M,0,155
4,adult,F,0,9207
5,adult,F,1,6269
6,adult,M,1,7944
7,adult,M,0,13235
8,neonate,F,0,3601
9,neonate,F,1,28
